In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import os
import torch
import open_clip
import numpy as np
import pandas as pd
from tqdm import tqdm
from PIL import Image
from timm import create_model
from torchvision import transforms
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [3]:
file_path = "/fs/ess/PAS2136/Hawaii-2025/beetles_intake/BeetlePalooza Data/Benchmarking/Beetlepalooza_beetles_image_only.csv"
df = pd.read_csv(file_path, sep="\t")
df = df[['ImageFilePath', 'ScientificName']]
df.head(2)

,ImageFilePath,ScientificName
0,/fs/ess/PAS2136/Rayeed/BeetlePalooza/individua...,Chlaenius aestivus
1,/fs/ess/PAS2136/Rayeed/BeetlePalooza/individua...,Chlaenius aestivus


In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = create_model("convnext_base", pretrained=True).to(device)


preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [5]:
def extract_image_features(image_path) : 
    
    image = Image.open(image_path).convert("RGB")
    
    image_tensor = preprocess(image).unsqueeze(0).to(device)

    with torch.no_grad():
        features = model.forward_features(image_tensor).mean(dim=[2, 3])

    return features.cpu().numpy()
    

In [6]:
X = np.vstack([extract_image_features(img) for img in tqdm(df["ImageFilePath"], desc="Processing ...") ])

print(f"Extracted feature shape: {X.shape}")


le = LabelEncoder()

y = le.fit_transform(df["ScientificName"])

df_indices = df.index 

X_train, X_test, y_train, y_test, train_idx, test_idx = train_test_split(X, y, df_indices, test_size=0.2, random_state=42)

test_df = df.loc[test_idx].copy()


Processing ...: 100%|██████████| 11399/11399 [22:07<00:00,  8.58it/s]

Extracted feature shape: (11399, 1024)


In [7]:
from sklearn.preprocessing import StandardScaler


scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)

X_test_scaled = scaler.transform(X_test)


In [8]:
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, balanced_accuracy_score, matthews_corrcoef


seed = 99


models = {
    "NaiveBayes": GaussianNB(),
    "LogisticRegression": LogisticRegression(max_iter=100),
    "SVMLinear": SVC(kernel="linear"),
    "SVMPolynomial": SVC(kernel="poly", degree=4),
    "SVMRadialBasis": SVC(kernel="rbf", degree=4),
    "NearestNeighbor": KNeighborsClassifier(n_neighbors=5),
    "RandomForest": RandomForestClassifier(n_estimators=100, random_state=seed),    
    "MLP_Baseline": MLPClassifier(hidden_layer_sizes=(256, 128), activation='tanh', max_iter=200, random_state=seed)
}

predictions = {}

metrics = {}


for name, model in models.items():
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    predictions[name] = preds

    acc = accuracy_score(y_test, preds)
    prec = precision_score(y_test, preds, average="weighted")
    rec = recall_score(y_test, preds, average="weighted")
    f1 = f1_score(y_test, preds, average="weighted")
    bal_acc = balanced_accuracy_score(y_test, preds)
    mcc = matthews_corrcoef(y_test, preds)

    metrics[name] = {"Model": name, "Accuracy": acc, "Precision": prec, "Recall": rec, "F1-Score": f1, "Balanced Acc": bal_acc, "MCC": mcc}

    print(f"{name:<25} | Acc: {acc:.2%} | Prec: {prec:.2%} | Rec: {rec:.2%} | F1: {f1:.2%} | Bal Acc: {bal_acc:.2%} | MCC: {mcc:.4f}")


metrics_df = pd.DataFrame(metrics).T


NaiveBayes                | Acc: 71.84% | Prec: 78.18% | Rec: 71.84% | F1: 73.55% | Bal Acc: 63.35% | MCC: 0.7049
LogisticRegression        | Acc: 93.68% | Prec: 93.08% | Rec: 93.68% | F1: 93.20% | Bal Acc: 75.67% | MCC: 0.9325
SVMLinear                 | Acc: 94.56% | Prec: 93.75% | Rec: 94.56% | F1: 93.98% | Bal Acc: 77.43% | MCC: 0.9419
SVMPolynomial             | Acc: 89.12% | Prec: 85.46% | Rec: 89.12% | F1: 86.58% | Bal Acc: 51.59% | MCC: 0.8836
SVMRadialBasis            | Acc: 86.32% | Prec: 81.49% | Rec: 86.32% | F1: 82.91% | Bal Acc: 44.24% | MCC: 0.8535
NearestNeighbor           | Acc: 83.20% | Prec: 81.88% | Rec: 83.20% | F1: 81.39% | Bal Acc: 56.43% | MCC: 0.8201
RandomForest              | Acc: 79.21% | Prec: 74.14% | Rec: 79.21% | F1: 75.11% | Bal Acc: 37.60% | MCC: 0.7773
MLP_Baseline              | Acc: 95.31% | Prec: 94.26% | Rec: 95.31% | F1: 94.61% | Bal Acc: 77.26% | MCC: 0.9498


In [9]:
test_df = test_df.assign(**{f"Pred_{name}": le.inverse_transform(pred) for name, pred in predictions.items()})
test_df.head(2)

,ImageFilePath,ScientificName,Pred_NaiveBayes,Pred_LogisticRegression,Pred_SVMLinear,Pred_SVMPolynomial,Pred_SVMRadialBasis,Pred_NearestNeighbor,Pred_RandomForest,Pred_MLP_Baseline
1766,/fs/ess/PAS2136/Rayeed/BeetlePalooza/individua...,Synuchus impunctatus,Calathus advena,Synuchus impunctatus,Calathus advena,Calathus advena,Calathus advena,Calathus advena,Calathus advena,Synuchus impunctatus
2629,/fs/ess/PAS2136/Rayeed/BeetlePalooza/individua...,Agonum punctiforme,Agonum punctiforme,Agonum punctiforme,Agonum punctiforme,Synuchus impunctatus,Synuchus impunctatus,Synuchus impunctatus,Calathus advena,Agonum punctiforme


In [10]:
test_df.to_csv("/users/PAS2136/rayees/3. Benchmarking/BeetlePalooza/15.ConvNeXt-linear-probing-species.csv", index=False)
metrics_df.to_csv("/users/PAS2136/rayees/3. Benchmarking/BeetlePalooza/15.ConvNeXt-linear-probing-species-metrics.csv", index=False)